# PEFT 库 LoRA 实战 - OpenAI Whisper-large-v2

In [1]:
import torch  # 导入 PyTorch 库

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    print(f"GPU 可用，当前 GPU 数量: {torch.cuda.device_count()}")  # 输出可用 GPU 的数量
    print(f"当前使用的 GPU: {torch.cuda.get_device_name(0)}")      # 输出当前使用的 GPU 名称
else:
    print("未检测到可用的 GPU")  # 没有检测到可用的 GPU 时输出

GPU 可用，当前 GPU 数量: 1
当前使用的 GPU: Quadro RTX 6000


In [2]:
import torch
print(torch.__version__)

2.7.0+cu128


In [3]:
import torch
print(torch.backends.cudnn.enabled)  # True 表示 cuDNN 可用

True


In [4]:
model_name_or_path = "openai/whisper-large-v2"
model_dir = "models/whisper-large-v2-asr-int8"

language = "Chinese (China)"
language_abbr = "zh-CN"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_13_0"

batch_size=4

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "parquet",
    data_files="/home/pactera/aistudy/week02/Lizhe/datasets/zh/train/*.parquet",
    split="train"
)

common_voice["validation"] = load_dataset(
    "parquet",
    data_files="/home/pactera/aistudy/week02/Lizhe/datasets/zh/validation/*.parquet",
    split="train"  # 这里用 train 标记，后续通过 DatasetDict 区分
)

# 验证加载结果
print("训练集大小：", len(common_voice["train"]))
print("训练集第一条数据：", common_voice["train"][0])
print("验证集大小：", len(common_voice["validation"]))

训练集大小： 29383
训练集第一条数据： {'client_id': 'bcb4464171113dd9b51f371c3eecea06771fde83e7e3239ad0516469c6dcdf80170d26c7d1b1ef2476c45b51bfb4ee5549f07d7002bcfcec9b371a30c873b92d', 'path': 'zh-CN_train_0/common_voice_zh-CN_18551060.mp3', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x7fb4ddad7550>, 'sentence': '巴顿是位于美国加利福尼亚州阿马多尔县的一个非建制地区。', 'up_votes': 2, 'down_votes': 0, 'age': 'twenties', 'gender': 'male', 'accent': '', 'locale': 'zh-CN', 'segment': '', 'variant': ''}
验证集大小： 10624


In [6]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # 检查是否支持 GPU

2.7.0+cu128
True


In [7]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 29383
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 10624
    })
})

In [8]:
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor

# 从预训练模型加载特征提取器
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

# 从预训练模型加载分词器，可以指定语言和任务以获得最适合特定需求的分词器配置
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

# 从预训练模型加载处理器，处理器通常结合了特征提取器和分词器，为特定任务提供一站式的数据预处理
processor = AutoProcessor.from_pretrained(model_name_or_path, language=language, task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

In [10]:
common_voice["train"][0]

{'audio': <datasets.features._torchcodec.AudioDecoder at 0x7fb45e3b6b50>,
 'sentence': '巴顿是位于美国加利福尼亚州阿马多尔县的一个非建制地区。',
 'variant': ''}

### 降采样音频数据

In [11]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
# sampling_rate 从 48KHZ 降为 16KHZ
common_voice["train"][0]

{'audio': <datasets.features._torchcodec.AudioDecoder at 0x7fb45e3b7790>,
 'sentence': '巴顿是位于美国加利福尼亚州阿马多尔县的一个非建制地区。',
 'variant': ''}

### 整合以上数据处理为一个函数

In [13]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

### 数据抽样（演示需要）

In [14]:
all_common_voice = DatasetDict()

all_common_voice["train"] = common_voice["train"].shuffle(seed=16)
all_common_voice["validation"] = common_voice["validation"].shuffle(seed=16)

In [15]:
all_common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 29383
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'variant'],
        num_rows: 10624
    })
})

In [16]:
# 抽样数据处理
tokenized_common_voice = all_common_voice.map(prepare_dataset)

# 完整数据训练，尝试开启 `num_proc=8` 参数多进程并行处理（如阻塞无法运行，则不使用此参数）
# tokenized_common_voice = common_voice.map(prepare_dataset, num_proc=8)

In [17]:
tokenized_common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', 'variant', 'input_features', 'labels'],
        num_rows: 29383
    })
    validation: Dataset({
        features: ['audio', 'sentence', 'variant', 'input_features', 'labels'],
        num_rows: 10624
    })
})

# 自定义语音数据整理器

In [18]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

# 定义一个针对语音到文本任务的数据整理器类
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # 处理器结合了特征提取器和分词器

    # 整理器函数，将特征列表处理成一个批次
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 从特征列表中提取输入特征，并填充以使它们具有相同的形状
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 从特征列表中提取标签特征（文本令牌），并进行填充
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 使用-100替换标签中的填充区域，-100通常用于在损失计算中忽略填充令牌
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 如果批次中的所有序列都以句子开始令牌开头，则移除它
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理过的标签添加到批次中
        batch["labels"] = labels

        return batch  # 返回最终的批次，准备好进行训练或评估

In [19]:
# 用给定的处理器实例化数据整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## 模型准备
### 加载预训练模型（int8 精度）
### 使用 int8 精度加载预训练模型，进一步降低显存需求。

In [20]:
from transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, load_in_8bit=False, device_map="auto" )

In [21]:
import torch
print(torch.__version__)

2.7.0+cu128


In [22]:
# 设置模型配置中的forced_decoder_ids属性为None
model.config.forced_decoder_ids = None  # 这通常用于指定在解码（生成文本）过程中必须使用的特定token的ID，设置为None表示没有这样的强制要求

# 设置模型配置中的suppress_tokens列表为空
model.config.suppress_tokens = []  # 这用于指定在生成过程中应被抑制（不生成）的token的列表，设置为空列表表示没有要抑制的token

# PEFT 微调前的模型处理

In [23]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

# 创建一个LoraConfig对象，用于设置LoRA（Low-Rank Adaptation）的配置参数
config = LoraConfig(
    r=4,  # LoRA的秩，影响LoRA矩阵的大小
    lora_alpha=64,  # LoRA适应的比例因子
    # 指定将LoRA应用到的模型模块，通常是attention和全连接层的投影。
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,  # 在LoRA模块中使用的dropout率
    bias="none",  # 设置bias的使用方式，这里没有使用bias
)

In [24]:
from peft import get_peft_model

model = get_peft_model(model,config)

# LoRA Adapter 配置

# 使用get_peft_model函数和给定的配置来获取一个PEFT模型

In [25]:
peft_model = get_peft_model(model, config)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'openai/whisper-large-v2' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [26]:
# 打印 LoRA 微调训练的模型参数
peft_model.print_trainable_parameters()

trainable params: 1,966,080 || all params: 1,545,271,040 || trainable%: 0.1272


### 模型训练
#### Seq2SeqTrainingArguments 训练参数
关于设置训练步数和评估步数

基于 epochs 设置：

num_train_epochs=3,  # 训练的总轮数
evaluation_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估
warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
基于 steps 设置：

max_steps=100, # 训练总步数
evaluation_strategy="steps", 
eval_steps=25, # 评估步数

In [27]:
from transformers import Seq2SeqTrainingArguments

# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批量大小
    learning_rate=1e-3,  # 学习率
    num_train_epochs=1,  # 训练的总轮数
    evaluation_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估
    # warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
    # fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批量大小
    generation_max_length=32,  # 生成任务的最大长度
    logging_steps=10,  # 指定日志记录的步骤，用于跟踪训练进度
    remove_unused_columns=False,  # 是否删除不使用的列，以减少数据处理开销
    label_names=["labels"],  # 指定标签列的名称，用于训练过程中
    # evaluation_strategy="steps",
    # eval_steps=25,
    fp16=True,  
    fp16_opt_level="O1", 
)

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 实例化 Seq2SeqTrainer 训练器

In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=peft_model,
    train_dataset=tokenized_common_voice["train"],
    eval_dataset=tokenized_common_voice["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor, 
)
peft_model.config.use_cache = False

/home/pactera/miniconda3/envs/aistudy5/lib/python3.11/site-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [29]:
!nvidia-smi

Fri Aug  1 17:37:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                Off |   00000000:B3:00.0  On |                  Off |
| 33%   43C    P0             68W /  260W |    7455MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.305700,0.361165


TrainOutput(global_step=7346, training_loss=0.5035409991630182, metrics={'train_runtime': 11653.8523, 'train_samples_per_second': 2.521, 'train_steps_per_second': 0.63, 'total_flos': 6.2468321937408e+19, 'train_loss': 0.5035409991630182, 'epoch': 1.0})

# 保存 LoRA 模型(Adapter)

In [31]:
trainer.save_model(model_dir)

In [32]:
peft_model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): PeftModel(
      (base_model): LoraModel(
        (model): WhisperForConditionalGeneration(
          (model): WhisperModel(
            (encoder): WhisperEncoder(
              (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
              (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
              (embed_positions): Embedding(1500, 1280)
              (layers): ModuleList(
                (0-31): 32 x WhisperEncoderLayer(
                  (self_attn): WhisperSdpaAttention(
                    (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
                    (v_proj): lora.Linear(
                      (base_layer): Linear(in_features=1280, out_features=1280, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lora_A): ModuleDict(
               

# 模型推理（可能需要重启 Notebook）
### 使用 PeftModel 加载 LoRA 微调后 Whisper 模型
### 使用 PeftConfig 加载 LoRA Adapter 配置参数，使用 PeftModel 加载微调后 Whisper 模型

In [1]:
model_dir = "models/whisper-large-v2-asr-int8"
model_name_or_path = "openai/whisper-large-v2"

language = "Chinese (China)"
language_abbr = "zh-CN"
language_decode = "chinese"
task = "transcribe"

In [2]:
from transformers import AutoModelForSpeechSeq2Seq, AutoTokenizer, AutoProcessor
from peft import PeftConfig, PeftModel

peft_config = PeftConfig.from_pretrained(model_dir)

In [3]:
peft_config.base_model_name_or_path = model_name_or_path

In [4]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'PeftModel', 'parent_library': 'peft.peft_model'}, base_model_name_or_path='openai/whisper-large-v2', revision=None, task_type=None, inference_mode=True, r=4, target_modules={'q_proj', 'v_proj'}, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [5]:
base_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=False, device_map="auto"
)

peft_model = PeftModel.from_pretrained(base_model, model_dir)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = AutoProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 使用 Pipeline API 部署微调后 Whisper 实现中文语音识别任务

In [7]:
test_audio = "zh_test.mp3"

In [8]:
from transformers import AutomaticSpeechRecognitionPipeline

pipeline = AutomaticSpeechRecognitionPipeline(model=peft_model, tokenizer=tokenizer, feature_extractor=feature_extractor)

forced_decoder_ids = processor.get_decoder_prompt_ids(language=language_decode, task=task)

In [9]:
import torch

with torch.cuda.amp.autocast():
    text = pipeline(test_audio, max_new_tokens=255)["text"]

/tmp/ipykernel_1066041/3500902550.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [10]:
text

'這是一個中文測試'